In [1]:
from sklearn.datasets import load_wine

X, y = load_wine(return_X_y=True)

In [15]:
import numpy as np
from scipy.spatial import KDTree
import cvxpy as cp

class DKNN:

    def __init__(self, k):
        self.k = k
        self.A = None
        self.pi = None
        self.trees = []

    def dist(self, x, mu, c):
        delta = x - mu
        return np.sum(np.multiply(delta @ self.A, delta), axis=-1) - self.pi[c]

    def fit(self, X, y, alpha=1, beta=1):
        self.X = X
        self.C = np.unique(y)
        self.c_idx = []
        for ci in self.C:
            self.c_idx.append(np.where(y == ci))
        n, d = X.shape

        centroids = []

        # Find centroids of class C[i]
        for idx in self.c_idx:
            # Get k nearest neighbors of class C[i] for all X
            tree = KDTree(X[idx])
            _, n_idx = tree.query(X, self.k)
            self.trees.append(tree)

            # Compute centroids
            neighbors = X[idx][n_idx]
            if self.k == 1:
                centroid_c = neighbors
            else:
                centroid_c = np.mean(neighbors, axis=1)
            centroids.append(centroid_c)
        
        centroids = np.stack(centroids, axis=0)

        # Convex problem formulation
        self.pi = np.array([len(idx[0]) / n for idx in self.c_idx])
        self.A = cp.Variable((d, d))

        delta = X - centroids

        # should work
        # f_mult = np.sum(np.multiply(delta @ self.A, delta), axis=2) - self.pi[:, np.newaxis]
        # print(f_mult[0, 0])

        constraints = []
        epsilon = cp.Variable(n)
        constraints.append(epsilon >= 0)

        for i in range(n):
            for c in self.C:
                if c == y[i]:
                    continue
                constraints += [
                    delta[y[i], i] @ self.A @ delta[y[i], i].T - cp.log(self.pi[y[i]]) + 1 - epsilon[i] <= delta[c, i] @ self.A @ delta[c, i].T - cp.log(self.pi[c])
                ]
            constraints += [
                epsilon[i] >= 0
            ]
        
        objective = cp.Minimize(cp.sum(alpha * epsilon) + beta * cp.norm(self.A))

        prob = cp.Problem(objective, constraints)
        prob.solve()

        self.A = self.A.value

    def predict(self, X_new):
        if X_new.ndim == 1:
            n = 1
        else:
            n = X_new.shape[0]
        dist_c = np.empty((n, len(self.trees)))
        for c, t in enumerate(self.trees):
            _, n_idx = t.query(X_new, self.k)

            # Compute centroids
            neighbors = self.X[self.c_idx[c]][n_idx]
            centroid = np.mean(neighbors, axis=-2)
            cur_dist = self.dist(X_new, centroid, c)
            dist_c[:, c] = cur_dist
        
        predictions = np.argmin(dist_c, axis=1)
        return predictions
    
    def score(self, X_test, y_test):
        y_pred = self.predict(X_test)
        return np.average(y_pred == y_test)

In [16]:
from sklearn.model_selection import ShuffleSplit

for k in range(1, 11):
    n_splits = 10
    total_score = 0
    rs = ShuffleSplit(n_splits=n_splits, test_size=.3)
    for i, (train_index, test_index) in enumerate(rs.split(X)):
        dknn = DKNN(k)
        dknn.fit(X[train_index], y[train_index])
        total_score += dknn.score(X[test_index], y[test_index])
    print(f"k = {k}: {total_score / n_splits}")

k = 1: 0.5796296296296297
k = 2: 0.8166666666666667
k = 3: 0.8055555555555557
k = 4: 0.875925925925926
k = 5: 0.837037037037037
k = 6: 0.8870370370370371
k = 7: 0.8944444444444445
k = 8: 0.8907407407407406
k = 9: 0.9055555555555556
k = 10: 0.875925925925926
